<a href="https://colab.research.google.com/github/fhenningsen/ai/blob/master/USE_Test_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Universal Sentence Encoder Test 2**

In [0]:
%%capture
# Install the latest Tensorflow version.
!pip3 install --upgrade tensorflow-gpu
!pip3 install --upgrade tensorflow_text
# Install TF-Hub.
!pip3 install tensorflow-hub
!pip3 install seaborn

!pip3 install nmslib

In [0]:
import tensorflow_hub as hub
import numpy as np
import tensorflow_text
from typing import Iterable, List

module_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5", "https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3 loaded


In [0]:
BATCH_SIZE = 32
def sanitize_input(sentence: str):
    return sentence.replace('\n', '').lower().strip()[:1000]

def encode(data: Iterable[str]) -> np.array:
    data = [sanitize_input(sentence) for sentence in data]
    all_vectors: List[List[float]] = []
        
    for i in range(0, len(data), BATCH_SIZE):
        batch = data[i:i+BATCH_SIZE]
        embeddings = embed(batch).numpy().tolist()
        all_vectors += embeddings
            
    return np.array(all_vectors).astype(np.float32)


In [0]:
import nmslib

data = [
    'What color is chameleon?',
    'When is the festival of colors?',
    'When is the next music festival?',
    'How far is the moon?',
    'How far is the sun?',
    'What happens when the sun goes down?',
    'What we do in the shadows?',
    'What is the meaning of all this?',
    'What is the meaning of Russel\'s paradox?',
    'How are you doing?'
]
encoded_data = encode(data)

# initialize a new index, using a HNSW index on Cosine Similarity
index = nmslib.init(method='hnsw', space='cosinesimil')
index.addDataPointBatch(encoded_data)
index.createIndex({'post': 2}, print_progress=True)

query = [
    'كم تبعد الشمس؟'
]

encoded_query = encode(query)

# query for the nearest neighbours of the first datapoint
ids, distances = index.knnQuery(encoded_query, k=3)

for i in range(3):
    print(data[ids[i]] + " (" + str(distances[i]) + ")") 


How far is the sun? (0.10776508)
How far is the moon? (0.27791917)
What happens when the sun goes down? (0.40345967)
